# Background 
**A Private Equity firm intends to diversify their industry presence and wishes to enter the growing Fast Food Market. They are on lookout to support Brands which are currently struggling by investing in digital capabilities. We wish to provide them with an assessment of current landscape and  learn top brands in marketplace. Apart from this we will also seek to know which brands are falling short of health code inspections and their ratings on Yelp**  


# Motivation
**I wanted to explore and draw insights from wide variety of data sources to answer larger questions at hand. So very often, looking at one kind of data to seek answers might bias the results or give us partial answers so it is vital that we keep an open mind for other datasets that might shed light on what we wish to know. This is ground reality for data scientists in their daily jobs. In addition I also wanted to improve upon my python and data visualization skills. I would greatly value your feedback and thoughts on overall analysis**

**Happy Learning!:--)**

**Disclaimer: This analysis is for educational purposes only and does not reflect the true picture. All the data sources employed are available on Kaggle**

# Objectives
**1. Overview of Fast Food Restaurant Landscape**  
**2. Analyze Brand Presence by Geographies**    
**3. Inspect Health Violations Data by Brand**   
**4. Review Sentiment of Brands on Yelp**  

**Import Libraries and Data**  
**In order to workout the analysis we are loading all the required datasets into our environment**


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import folium
from folium.plugins import FastMarkerCluster
import re

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

pd.set_option('display.max_rows', 500)

/kaggle/input/yelp-dataset/yelp_academic_dataset_checkin.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_user.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_tip.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_business.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_review.json
/kaggle/input/yelp-dataset/Dataset_Agreement.pdf
/kaggle/input/fast-food-restaurants/Datafiniti_Fast_Food_Restaurants_May19.csv
/kaggle/input/fast-food-restaurants/FastFoodRestaurants.csv
/kaggle/input/fast-food-restaurants/Datafiniti_Fast_Food_Restaurants.csv
/kaggle/input/la-restaurant-market-health-data/socrata_metadata_restaurant-and-market-health-violations.json
/kaggle/input/la-restaurant-market-health-data/restaurant-and-market-health-violations.csv
/kaggle/input/la-restaurant-market-health-data/socrata_metadata_restaurant-and-market-health-inspections.json
/kaggle/input/la-restaurant-market-health-data/restaurant-and-market-health-inspections.csv
/kaggle/input/chicago-food-inspect

**Below Dataset is a sample of 10000 restaurants located in US which is provided by Datafiniti. We may consider this dataset to be representative of broader market and draw our conclusions**

In [2]:
#Import the dataset and keep relevant columns only
data=pd.read_csv("/kaggle/input/fast-food-restaurants/FastFoodRestaurants.csv")
del data['websites']
data.drop(columns=['keys'],inplace=True)
data.head(5)


,address,city,country,latitude,longitude,name,postalCode,province
0,324 Main St,Massena,US,44.92130,-74.89021,McDonald's,13662,NY
1,530 Clinton Ave,Washington Court House,US,39.53255,-83.44526,Wendy's,43160,OH
2,408 Market Square Dr,Maysville,US,38.62736,-83.79141,Frisch's Big Boy,41056,KY
3,6098 State Highway 37,Massena,US,44.95008,-74.84553,McDonald's,13662,NY
4,139 Columbus Rd,Athens,US,39.35155,-82.09728,OMG! Rotisserie,45701,OH


**We see that province appears to be State and we have latitude and longitude details along with the address. We shall break up over 50 odd states into key regions to learn Brand strenghs at regional level. First lets take a closer look at dataset by plotting them on Map**

In [3]:
lats = data['latitude'].tolist()
lons = data['longitude'].tolist()
locations = list(zip(lats, lons))

map1 = folium.Map(location=[37.0902, -95.7129], zoom_start=5)
FastMarkerCluster(data=locations).add_to(map1)
map1

**We see presence across all the states with California, Florida and New York leading the way. Now lets split the entire market into 4 key regions**

In [4]:
def market(x):
    
    if x in ('WA','OR','CA','NV','AZ','NM','CO','WY','MT','ID','AK','HI'):
        return "West"
    elif x in ('ND','MN','IA','SD','KS','NE','MO','WI','MI','IL','IN','OH'):
        return "Mid West"
    elif x in ('NY','NJ','PA','CT','RI','MA','ME','NH','VT'):
        return "North East"
    else:
        return "South"

data['region']=data['province'].apply(market)
data.head(5)

,address,city,country,latitude,longitude,name,postalCode,province,region
0,324 Main St,Massena,US,44.92130,-74.89021,McDonald's,13662,NY,North East
1,530 Clinton Ave,Washington Court House,US,39.53255,-83.44526,Wendy's,43160,OH,Mid West
2,408 Market Square Dr,Maysville,US,38.62736,-83.79141,Frisch's Big Boy,41056,KY,South
3,6098 State Highway 37,Massena,US,44.95008,-74.84553,McDonald's,13662,NY,North East
4,139 Columbus Rd,Athens,US,39.35155,-82.09728,OMG! Rotisserie,45701,OH,Mid West


**Lets dive into the dataset**  

# 1. Overview of Fast Food Restaurant Landscape 
**Lets obtain presence of restaurants at Regional level**

In [5]:
d1=data.groupby(['region','province']).agg({'name':'count'})
d1.columns=['Perc_Region']
d1=d1.groupby(level=0).apply(lambda x: 100*x/x.sum())
d1.reset_index(inplace=True)
d1['Perc_Region']=d1['Perc_Region'].round(1)


d1.sort_values(['region','Perc_Region'],ascending=[False,False],inplace=True)
d1.set_index(['region','province'])
d1.reset_index(inplace=True)

d11=d1[d1.region.isin(['West'])].sort_values(['Perc_Region'],ascending=True)
d12=d1[d1.region.isin(['Mid West'])].sort_values(['Perc_Region'],ascending=True)
d13=d1[d1.region.isin(['North East'])].sort_values(['Perc_Region'],ascending=True)
d14=d1[d1.region.isin(['South'])].sort_values(['Perc_Region'],ascending=True)

# d1.head(5)

fig=px.bar(d1,x=d1['province'],y=['Perc_Region'],color='region',title="Fast Food Restaurant Presence (%): Percentages are at Regional Level and not Overall")
fig.update_layout(xaxis_title="State",yaxis_title="Percentage")
fig.show()

**1. In West, California makes up 36% of total restaurants , Washington and Arizona are a long way off with about 11% share**   
**2. Texas (14.6%), Florida(10.9%) are top two markets in South while Pennsylvania(26%), New York(25%) lead in North East**   
**3. In Mid West, Ohio has highest among all with about 20%**  

**Now an overall view as well of fast food restaurants**

In [6]:
state_counts=data.groupby(['province']).agg({'name':'count'})
state_counts.columns=['Count of Brands']
state_counts.sort_values('Count of Brands',ascending=False,inplace=True)
state_counts.reset_index(inplace=True)

state_counts['Perc']=100*state_counts['Count of Brands']/state_counts['Count of Brands'].sum()
# state_counts.head(10)

fig=px.bar(state_counts,x="province",y="Perc")
fig.update_traces(marker_color='Turquoise')
fig.update_layout(xaxis_title="State",yaxis_title="Percentage",title="Fast Food Restaurant Presence(%)")
fig.show()

**1. California (6.8%), Texas(6.3%), Ohio(5.4%), Florida(4.7%), Indiana(3.8%) are top 5 markets in US**  
**2. Montana, Rikers Island, Alaska, Colorado Springs rank least**

**Lets examine market share by Region**

In [7]:
region_counts=data.groupby(['region']).agg({'name':'count'})
region_counts.columns=['Count of Brands']

region_counts.sort_values('Count of Brands', ascending=False,inplace=True)
region_counts['perc']=100*region_counts['Count of Brands']/region_counts['Count of Brands'].sum()
region_counts.reset_index(inplace=True)
# print(region_counts)

fig=px.pie(region_counts,values='perc',names='region',title='Brand Presence by Region',width=600,height=500,
           color_discrete_sequence=px.colors.sequential.RdBu)
fig.show()

**With 43.4% of fast restaurants located South takes the top spot while North East contains only 10.8%. For Brands looking to enter the market or expand their footprint, South and Mid West are favourable**

# 2. Analyze Brand Presence by Geographies
**First lets obtain an idea of top brands in the dataset**

In [8]:

data['name']=data.name.str.replace('$','')
data['name']=data.name.str.replace("'",'')
data['name']=data.name.str.lower()    
data['name']=data.name.map(lambda x: re.sub(r'\W+','',x))


Brand_1=data.groupby(['name']).agg({'name':'count'})
Brand_1.columns=['number']
Brand_1=Brand_1.sort_values('number',ascending=False)

# Brand_1.head(10)
Brand_1.nlargest(15,'number').index

Index(['mcdonalds', 'burgerking', 'tacobell', 'wendys', 'arbys', 'kfc',
       'subway', 'sonicdrivein', 'dominospizza', 'jackinthebox', 'hardees',
       'jimmyjohns', 'chickfila', 'pizzahut',
       'bojanglesfamouschickennbiscuits'],
      dtype='object', name='name')

**We are going to focus on top 7 brands only. We will filter out them from main dataset to inform our further analysis**

In [9]:
Brands=['mcdonalds', 'burgerking', 'tacobell', 'wendys', 'arbys', 'kfc','subway']
data1=data[data.name.isin(Brands)]

# print(data1.shape)
data1.head(10)

d1=data1.groupby(['region','name']).agg({'name':'count'})
d1.columns=['Stores']
d1.reset_index(inplace=True)
d1.sort_values(['region','Stores'],ascending=['True','False'],inplace=True)
d1.reset_index(inplace=True)
d1.drop('index',inplace=True,axis=1)

import plotly.graph_objects as go
import plotly.express as px

fig = px.scatter(d1, x="Stores", y="name", color="region",
                 title="Brand Presence by Region-Store Counts",
                 labels={"Stores":"Store Count"} 
                )

fig.update_layout(
    margin=dict(l=20, r=20, t=25, b=20)
#     paper_bgcolor="LightSteelBlue",
)
fig.show()

**McDonalds, Burger King, Taco Bell, Wendys, Arbys, KFC and Subway are the most popular Brands. Clearly all the Brands have big presence in South followed by Mid West. There are a few exceptions when compared to overall level. Burger King has more stores in North East than West which tells us that it is priority market for them**

**Lets obtain Brand Penetration at Region to confirm the above view**

In [10]:
d2=d1.copy()
d2.set_index(['region','name'],inplace=True)
d2['Percentage']=d2['Stores'].groupby(level=0).apply(lambda x: 100*x/x.sum())
# del d2['perc']
d2.reset_index(inplace=True)

# d2.reset_index(inplace=True)
fig2=px.histogram(d2,x="region",y="Percentage",color="name",width=800,height=500,
                 title="Brand Presence by Region")
# fig2.update_layout(margin=dict(l=))
fig2.show()

**1. McDonalds market penetration is above 30% in all regions with highest of 36% in North East**  
**2. Burger King has higher penetration in North East after McDonalds. Subway is concentrated more in West while Arbys dominates in MidWest**  
**3. Taco Bell has least presence in North East**

**Lets now obtain further granular insights of Brand Penetration by State level**

In [11]:
d3=data1.groupby(['province','name']).agg({'name':'count'})
d3.columns=['Stores']
d3['Percentage']=d3['Stores'].groupby(level=0).apply(lambda x:100*x/x.sum())
d3.reset_index(inplace=True)
# d3

fig4=px.histogram(d3,x="province",y="Percentage",color="name",
                 title="Brand Penetration by State")
fig4.show()

**Analyzing by state we learn the top markets for each brand. We observe that they are present in most of the markets. In Alaska only 4 brands are present**

**For McDonalds, Vermont, Texas, Connecticut are key ones. Wendys is present in all states except Delaware and Colorado Springs**
**Arbys is absent in DC, Mississipi, Rikers Island and Vermont, while KFC lacks any stores in Montana**

# 3. Inspect Health Violations Data by Brand

**We have so far gathered a clear understanding of Fast Foods Restaurants presence across US and as well as learned the geographic footprint of top brands. To aid our understanding of knowing which brands are facing difficulties in running their business so that our PE firm can step in and support, we are going to find out if any health code standards are being broken**

**Let us examine Los Angeles and Chicago for this purpose. Both are large cities with GDPs equivalent of some of the european countries. For Ex Chicago's GDP (611 Billion) is lot higher than Sweden's ($529 Billion)**

## Los Angeles City, CA
This dataset contains health inspection results for restaurants in Los Angeles City. The source for this dataset is provided by County of LA Public Health. Each row in this dataset is a health code violation  
https://data.lacity.org/A-Safe-City/Restaurant-and-Market-Health-Violations/ckya-qgys

In [12]:
dh=pd.read_csv('/kaggle/input/la-restaurant-market-health-data/restaurant-and-market-health-violations.csv')
# dh.columns
cols=['facility_name', 'violation_code',
       'violation_description', 'violation_status', 'points', 'grade',
       'facility_address', 'facility_city', 'facility_id', 'facility_state',
       'facility_zip','score', 'service_code',
       'service_description', 'row_id']
dh=dh[[x for x in cols]]
dh.drop_duplicates(keep=False,inplace=True)
# print(dh.shape)
dh.shape

(313675, 15)

In [13]:
# View top 5 records 
dh.head(5)

,facility_name,violation_code,violation_description,violation_status,points,grade,facility_address,facility_city,facility_id,facility_state,facility_zip,score,service_code,service_description,row_id
0,KRUANG TEDD,F030,# 30. Food properly stored; food storage conta...,OUT OF COMPLIANCE,1,A,5151 HOLLYWOOD BLVD,LOS ANGELES,FA0064949,CA,90027,92,1,ROUTINE INSPECTION,DA08R0TCUF030
1,KRUANG TEDD,F027,# 27. Food separated and protected,OUT OF COMPLIANCE,1,A,5151 HOLLYWOOD BLVD,LOS ANGELES,FA0064949,CA,90027,92,1,ROUTINE INSPECTION,DA08R0TCUF027
2,KRUANG TEDD,F035,# 35. Equipment/Utensils - approved; installed...,OUT OF COMPLIANCE,1,A,5151 HOLLYWOOD BLVD,LOS ANGELES,FA0064949,CA,90027,92,1,ROUTINE INSPECTION,DA08R0TCUF035
3,KRUANG TEDD,F033,# 33. Nonfood-contact surfaces clean and in go...,OUT OF COMPLIANCE,1,A,5151 HOLLYWOOD BLVD,LOS ANGELES,FA0064949,CA,90027,92,1,ROUTINE INSPECTION,DA08R0TCUF033
4,KRUANG TEDD,F029,"# 29. Toxic substances properly identified, st...",OUT OF COMPLIANCE,1,A,5151 HOLLYWOOD BLVD,LOS ANGELES,FA0064949,CA,90027,92,1,ROUTINE INSPECTION,DA08R0TCUF029


In [14]:
# Cleaning up facility names by removing special characters, white spaces and converting to lower
dh['facility_name']=dh.facility_name.apply(lambda x: re.sub(r'\W+','',x))
dh['facility_name']=dh.facility_name.str.lower()
dh[['facility_name','violation_code','violation_description']].head(5)

,facility_name,violation_code,violation_description
0,kruangtedd,F030,# 30. Food properly stored; food storage conta...
1,kruangtedd,F027,# 27. Food separated and protected
2,kruangtedd,F035,# 35. Equipment/Utensils - approved; installed...
3,kruangtedd,F033,# 33. Nonfood-contact surfaces clean and in go...
4,kruangtedd,F029,"# 29. Toxic substances properly identified, st..."


**What are the top health violations?**

In [15]:
# Obtain Top Health Violations 
dh1=dh['violation_description'].value_counts().sort_values(ascending=True).to_frame()
dh1.reset_index(inplace=True)
dh1.columns=['Violation Description','Number of Violations']
# dh1.tail(10)

fig11=px.bar(dh1[(dh1.shape[0]-10):],x='Number of Violations',y='Violation Description',orientation='h',height=500,width=1200)
fig11.update_layout(xaxis_title="Number of Violations",yaxis_title="Violation Code",title="Common Health Violations in LA")
fig11.update_traces(marker_color='darkcyan')
fig11.show()

**Common Violations appear to be related to Floors, Walls or Non Food contact surfaces lacking cleanliness. Plumbing, Ventilation, Storage of foods/containers are also among the top ones which are frequently cited for violations**

### Obtain violations by Brand 

In [16]:
dh2=dh['facility_name'].value_counts().sort_values(ascending=False).to_frame()
dh2.reset_index(inplace=True)
dh2.columns=['facility_name','Nbr of Violations']
# print(f'There are a total of {len(dh2)} Unique Facilities in dataset')

# selecting focus brands only
brands=['burgerki','donalds','subway','tacobell','kfc','arbys','wendys']
# dh2[dh2['facility_name'].str.contains(r'burgerking|donalds')]
dh2=dh2[dh2['facility_name'].str.contains('|'.join(map(re.escape, brands)))]
# dh2.head(5)

# we observe that brand names are combined with numeric data henceforth further refining
dh2.loc[dh2['facility_name'].str.contains(r'burgerking'),'Brand']="BurgerKing"
dh2.loc[dh2['facility_name'].str.contains(r'wendys'),'Brand']="Wendys"
dh2.loc[dh2['facility_name'].str.contains(r'mcdonalds'),'Brand']="McDonalds"
dh2.loc[dh2['facility_name'].str.contains(r'subway'),'Brand']="Subway"
dh2.loc[dh2['facility_name'].str.contains(r'kfc'),'Brand']="KFC"
dh2.loc[dh2['facility_name'].str.contains(r'tacobell'),'Brand']="TacoBell"
dh2.loc[dh2['facility_name'].str.contains(r'arbys'),'Brand']="Arbys"

# group at brand level

dh3=dh2[['Brand','Nbr of Violations','facility_name']].groupby(['Brand']).agg({'Nbr of Violations':'sum',
                                'facility_name':'count'})
dh3.reset_index(inplace=True)
dh3.columns=['Brands','Nbr_of_Violations','Stores']
# Normalizing Violations across Brands
dh3['Violations_per_store']=dh3['Nbr_of_Violations']/dh3['Stores']
dh3.sort_values('Violations_per_store',ascending=False,inplace=True)

fig5=make_subplots(rows=1,cols=3,subplot_titles=("Total Violations","Store Counts","Violations Per Store"))

trace_1=go.Bar(x=dh3.Brands,y=dh3.Nbr_of_Violations,name='Total Violations')
trace_2=go.Bar(x=dh3.Brands,y=dh3.Stores,name='Store Counts')
trace_3=go.Bar(x=dh3.Brands,y=dh3.Violations_per_store,name='Violations Per Store')

fig5.append_trace(trace_1,1,1)
fig5.append_trace(trace_2,1,2)
fig5.append_trace(trace_3,1,3)

fig5.update_layout(title='Health Inspection Results by Brands- Los Angeles',showlegend=False,height=450)

fig5.show()

**If a brand has more stores than their competitors then it is likely to have more violations registered. Henceforth we also would like to know number of stores for which the inspections were carried out by City**

**1. Subway, McDonalds lead in terms of total violations. This can be explained in part due to higher store counts**  
**2. At Violations per store, KFC records 53 while McDonalds, Subway and others are far lower at 24, 21 respectively**  
**3. TacoBell and BurgerKing have lowest violations**  

**In Los Angeles City, it appears KFC is in hot waters. Burger King and TacoBell despite having higher stores than Wendys and Arbys register fewer violations**


**Lets carry out a similar analysis for another big city**

## Chicago City, IL

In [17]:
df=pd.read_csv('/kaggle/input/chicago-food-inspections/food-inspections.csv')
df.drop_duplicates(keep=False,inplace=True)
df.columns

Index(['Inspection ID', 'DBA Name', 'AKA Name', 'License #', 'Facility Type',
       'Risk', 'Address', 'City', 'State', 'Zip', 'Inspection Date',
       'Inspection Type', 'Results', 'Violations', 'Latitude', 'Longitude',
       'Location', 'Historical Wards 2003-2015', 'Zip Codes',
       'Community Areas', 'Census Tracts', 'Wards'],
      dtype='object')

In [18]:
df=df[['DBA Name', 'AKA Name', 'Facility Type','Address',
       'Risk', 'City', 'State', 'Zip', 'Inspection Date',
       'Inspection Type', 'Results','Violations']]
df.head(5)

,DBA Name,AKA Name,Facility Type,Address,Risk,City,State,Zip,Inspection Date,Inspection Type,Results,Violations
0,CHIPOTLE MEXICAN GRILL,CHIPOTLE MEXICAN GRILL,Restaurant,1025 W ADDISON ST,Risk 1 (High),CHICAGO,IL,60613.0,2019-12-04T00:00:00.000,License Re-Inspection,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E..."
1,CHILI'S T-I,CHILI'S (T1-B14),Restaurant,11601 W TOUHY AVE,Risk 1 (High),CHICAGO,IL,60666.0,2019-12-04T00:00:00.000,Canvass,Pass,10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...
2,NICK'S FIRE GRILL STEAK & LEMONADE INC.,NICK'S FIRE GRILL STEAK & LEMONADE INC.,NaN,2900 W 63RD ST,Risk 1 (High),CHICAGO,IL,60629.0,2019-12-04T00:00:00.000,License,Not Ready,NaN
3,TAQUERIA BLUE LINE INC.,NaN,Restaurant,3401 W BELMONT AVE,Risk 1 (High),CHICAGO,IL,60618.0,2019-12-04T00:00:00.000,License,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E..."
4,PORTAGE PARK DAY NURSERY,MOSAIC EARLY CHILDHOOD ACADEMY,Children's Services Facility,5332-5334 W ADDISON ST,Risk 1 (High),CHICAGO,IL,60641.0,2019-12-04T00:00:00.000,Canvass,Pass,NaN


In [19]:
print(f'the violations description is very specific. There are {len(df.Violations.unique())} unique violations')

the violations description is very specific. There are 143498 unique violations


In [20]:
# Since Violation descriptions are unique we are selecting only first 75 characters from the string

df['Violations2']=df['Violations'].str[:75]
df.drop('Violations',1,inplace=True)

df['DBA Name']=df['DBA Name']+df['Address']
df=df[df['Results']=='Fail']
df1=df[['Violations2']].groupby(['Violations2']).agg({'Violations2':'count'})
df1.columns=['Counts']
df1.reset_index(inplace=True)
df1.columns=['Violations','Counts']
df1.sort_values('Counts',ascending=True,inplace=True)
# df1.head(10)

fig6=px.bar(df1[df1.shape[0]-10:],x='Counts',y='Violations',orientation='h',height=500,width=1500)
fig6.update_layout(xaxis_title="Number of Violations",yaxis_title="Violation Code",title="Common Health Violations in Chicago")
fig6.update_traces(marker_color='darkcyan')
fig6.show()

**Rodents, Insects presence is the main code which is often cited. Protection of food, Sanitizing, Dish/Hand washing are among the top10**

**Lets now look at individual brand level to see how they are faring**

In [21]:
df2=df.copy()
df2['DBA Name']=df2['DBA Name'].str.lower()
df2['DBA Name']=df2['DBA Name'].apply(lambda x: re.sub(r'\W+','',x))

df2=df2['DBA Name'].value_counts().sort_values(ascending=False).to_frame()
df2.reset_index(inplace=True)
df2.columns=['DBA Name','Nbr of Violations']
# print(f'There are a total of {len(dh2)} Unique Facilities in dataset')


brands=['burgerki','donalds','subway','tacobell','kfc','arbys','wendys']
df2=df2[df2['DBA Name'].str.contains('|'.join(brands))]

df2.loc[df2['DBA Name'].str.contains(r'burgerk'),'Brand']='BurgerKing'
df2.loc[df2['DBA Name'].str.contains(r'donalds'),'Brand']='McDonalds'
df2.loc[df2['DBA Name'].str.contains(r'arby'),'Brand']='Arbs'
df2.loc[df2['DBA Name'].str.contains(r'subway'),'Brand']='Subway'
df2.loc[df2['DBA Name'].str.contains(r'tacobe'),'Brand']='TacoBell'
df2.loc[df2['DBA Name'].str.contains(r'wendy'),'Brand']='Wendys'
df2.loc[df2['DBA Name'].str.contains(r'kfc'),'Brand']='KFC'
# df2.head(10)

df3=df2[['Brand','Nbr of Violations','DBA Name']].groupby(['Brand']).agg({'Nbr of Violations':'sum',
                                'DBA Name':'count'})
df3.reset_index(inplace=True)
df3.columns=['Brands','Nbr_of_Violations','Stores']

# Normalizing Violations across Brands
df3['Violations_per_store']=df3['Nbr_of_Violations']/df3['Stores']
df3.sort_values('Violations_per_store',ascending=False,inplace=True)
# df3.head(5)

fig7=make_subplots(rows=1,cols=3,subplot_titles=("Total Violations","Store Counts","Violations Per Store"))

trace_1=go.Bar(x=df3.Brands,y=df3.Nbr_of_Violations,name='Total Violations')
trace_2=go.Bar(x=df3.Brands,y=df3.Stores,name='Store Counts')
trace_3=go.Bar(x=df3.Brands,y=df3.Violations_per_store,name='Violations Per Store')

fig7.append_trace(trace_1,1,1)
fig7.append_trace(trace_2,1,2)
fig7.append_trace(trace_3,1,3)

fig7.update_layout(title='Health Inspection Results by Brands- Chicago',showlegend=False,height=450)

fig7.show()

**Subway, McDonalds once again fall short of required health standards. Total Violations are high due to presence of many stores. McDonalds, Burger King, KFC lead the fray in failing inspections**

## Summary of Health Inspection Data
**1. For both Chicago and Los Angeles cities, KFC and McDonalds are among the top 3 brands facing challenges in running their operations. They may already be incurring stiff penalities and repeated offences may cause them to lose their licenses**  

**2. McDonalds due to their larger presence in market, could have their troubles tied to other factors**

**3. KFC despite having much fewer stores is unable to meet necessary guidelines and may be in real trouble**

# 4. Review Sentiment of Brands on Yelp

**Yelp Ratings Analysis**  
**Let us now analyze user ratings left on Yelp to guage how users percieve food quality, customer service among others for each Brand**  

In [22]:
# Import json data into dataframe format and consider focus brands
b1= pd.read_json("/kaggle/input/yelp-dataset/yelp_academic_dataset_business.json", lines=True, orient='columns', chunksize=1000000)
for x in b1:
    b2 = x
    break

b3=b2.copy()
b3['name']=b3.name.str.lower()
b3['name']=b3['name'].apply(lambda x: re.sub('\W+','',x))
brands=['burgerking','kfc','wendys','arbys','mcdonalds','subway','tacobell']
b4=b3[b3['name'].str.match("|".join(brands))]

#Define new column as Brand
b4.loc[b4['name'].str.contains(r'wendys'),'Brand']='Wendys'
b4.loc[b4['name'].str.contains(r'burgerk'),'Brand']='Burgerking'
b4.loc[b4['name'].str.contains(r'kfc'),'Brand']='KFC'
b4.loc[b4['name'].str.contains(r'tacobe'),'Brand']='TacoBell'
b4.loc[b4['name'].str.contains(r'subway'),'Brand']='Subway'
b4.loc[b4['name'].str.contains(r'arbys'),'Brand']='Arbys'
b4.loc[b4['name'].str.contains(r'donalds'),'Brand']='McDonalds'

b4['Market']=b4['state'].apply(market)
b4.head(5)

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,Brand,Market
49,tLpkSwdtqqoXwU0JAGnApw,wendys,4602 Northfield Road,Cleveland,OH,44128,41.434614,-81.527026,3.5,7,1,"{'RestaurantsDelivery': 'False', 'BusinessAcce...","Restaurants, Fast Food, Burgers","{'Monday': '10:0-3:0', 'Tuesday': '10:0-3:0', ...",Wendys,Mid West
101,MTx-Zdl_KcU_z9G832XAjg,tacobell,8033 N Durango Dr,Las Vegas,NV,89131,36.307495,-115.279059,2.5,37,1,"{'OutdoorSeating': 'False', 'Alcohol': 'u'none...","Fast Food, Restaurants",None,TacoBell,West
250,ypILNgy7QFskKAdcPKB2RQ,kfc,1667 Ebenezer Rd,Rock Hill,SC,29732,34.956131,-81.046257,2.0,5,1,"{'GoodForKids': 'True', 'WiFi': ''free'', 'Res...","Restaurants, Fast Food, Chicken Shop, Chicken ...","{'Monday': '0:0-0:0', 'Tuesday': '10:0-22:0', ...",KFC,South
280,CfwrsG76Wm4iLS22v_wAcg,mcdonalds,6421 Pearl Rd,Parma Heights,OH,44130,41.385949,-81.768113,2.5,15,1,"{'Ambience': '{'romantic': False, 'intimate': ...","Restaurants, Coffee & Tea, Burgers, Fast Food,...","{'Monday': '6:0-23:0', 'Tuesday': '6:0-23:0', ...",McDonalds,Mid West
283,_V4CzzA7Z9h4qyLIdG-KUg,wendys,5214 Sunset Rd,Charlotte,NC,28269,35.307994,-80.841429,1.5,18,1,"{'RestaurantsTakeOut': 'True', 'GoodForKids': ...","Restaurants, Hot Dogs, Burgers, Fast Food",None,Wendys,South


**Lets see distribution of Ratings by brands**

In [23]:
fig8=px.histogram(b4,x='stars',color='Brand',facet_col='Brand',height=450)
fig8.update_layout(showlegend=False,title='Yelp Ratings')
fig8.show()

**Going by shape of distributions, Brands like McDonalds, KFC and Wendys skew lower ratings than others**

**To have a better understanding lets look at proportion of ratings at Brand level**

In [24]:
b5=b4.groupby(['Brand','stars']).agg({'name':'count'})
b5=b5.groupby(level=0).apply(lambda x: 100*x/x.sum())
b5.reset_index(inplace=True)
b5.columns=['Brand','stars','Perc']
b5['Perc']=b5['Perc'].round(1)

b5['stars']=b5['stars'].astype(str)
fig9=px.bar(b5,x='Brand',y='Perc',color='stars',labels={'stars':'Ratings','Perc':'% of Ratings'},
            color_continuous_scale=px.colors.sequential.Viridis,height=450,width=1100)
fig9.update_layout(title="Ratings Distribution")
fig9.show()

**1. If we consider a rating of 3 (Orange) as Neutral and use it as benchmark to measure amount of positive and negative ratings, we realize that Burger King, KFC and McDonalds contain high amount of less favourable reviews**  

**2. Arbys is rated much better than all other brands**




# Summary

## Market Landscape
* **Most of the fast food restaurants are located in South and Mid West. Further in each of the four regions few states have higher market share than others by large margin. Apart from having large urban centers, favourable business environment may also be factor for ex: lower taxes, state grants in establishing new stores, less stringent labor laws etc**  

* **At National level, California, Texas, Ohio, Florida and Indiana are top states attracting most of the businesses**

## Brand Presence
* **Top Brands have presence in almost all geographies with McDonalds having more stores than others by a considerable margin. This tells us that key growth strategy for Brands has been "Market Development" i.e offer products/services in newer markets as a way to grow revenues**  
* **McDonalds share is above 30% in all regions followed by Burger King at 17-24%**
* **Given the ubiquitous presence of brands, there is intense stiff competition in market. This creates added pressure on Brands to differentiate their product offerings which opens up opportunities for Brands to leverage their strengths while others face headwinds if they don't adapt in time. In market charecterized by high volume-low margin transactions mere survival may not be best long term strategy**

## Health Inspection Results
* **Lack of Ventilation, Cleanliness of non food surfaces, food storage and maintenance, Insects and Rodents presence are health codes which are violated often**   

* **In Los Angeles City KFC, McDonalds and Subway are responsible for most violations per store while in Chicago it's McDonalds, Burger King and KFC which fall foul of standards** 
* **McDonalds and KFC are both failing to meet set standards indicating that they are having trouble in managing their store operations. In addition they may be facing risk of store closures if this trend persists**  

* **Frequent fines and penalities erode profit margins and can hurt business**

## Yelp Rating Analysis
* **McDonalds, Burger King and KFC have many negative reviews relative to positive ones suggesting that customers are quite dissapointed with their service**  
* **Further analysis of reviews at store level should highlight key problems affecting the business**
* **This also shows that issues plaguing in running day to day store operations are having an affect on customer experience which may sound the death knell of Brand if not controlled immediately**

